In [53]:
import pandas as pd 
import numpy as np
import re 

In [ ]:
data = pd.read_csv("data/raw/fifa21_raw_data_v2.csv")

In [55]:
data.head()

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,...,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,...,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,...,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,...,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,...,High,Medium,5 ★,91,85,86,94,36,59,595


1. Convert the height and weight columns to numerical forms

In [56]:
def parse_height_to_metric(height: str):
    match = re.match(r"(\d+)'(\d+)\"", height)
    if match:
        feet = int(match.group(1))
        inches = int(match.group(2))
        total_inches = feet*12 + inches
        return total_inches * 2.54
    else:
        return int(height)
    
def parse_weight_to_metric(weight: str):
    match = re.match(r"(\d+)lbs", weight)
    if match:
        pounds = int(match.group(1))
        return pounds * 0.453592
    else:
        return int(weight)
        
data['Height'] = data['Height'].str.replace('cm','').apply(parse_height_to_metric)
data['Weight'] = data['Weight'].str.replace('kg', '').apply(parse_weight_to_metric)

2. Remove the unnecessary newline characters from all column that have them

In [57]:
def remove_newlines(text):
    if isinstance(text, str):
        return text.strip()
    else:
        return text

data = data.map(remove_newlines)

3. 'Value', 'Wage' and "Release Clause' are string columns. Convert them to numbers. For eg, "M" in value column is Million, so multiply the row values by 1,000,000, etc.

In [58]:
def convert_money_to_int(value: str):
    match_M = re.match(r'€\d+(\.\d+)?M', value)
    if match_M:
        value = value.replace('M', '')
        value = value.replace('€', '')
        value = float(value) * 10**6
    else:
        value = value.replace('K', '')
        value = value.replace('€', '')
        value = float(value) * 10**3
    
    return int(value)

data['Value'] = data['Value'].apply(convert_money_to_int)
data['Wage'] = data['Wage'].apply(convert_money_to_int)
data['Release Clause'] = data['Release Clause'].apply(convert_money_to_int)